In [1]:
import numpy as np
from sklearn.neighbors import KDTree

In [2]:
class Node:
    def __init__(self, nid: str, x: str, y: str, z:str):
        self.nid = int(nid)
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
        self.coordinates = np.array([self.x,self.y,self.z])
        
class ShellElement:
    def __init__(self, eid: str):
        self.eid = int(eid)
        self.attached_nodes = []
        
    def face_centroid(self, nodes):
        centroid = np.array([0.,0.,0.])
        
        for nid in self.attached_nodes:
            centroid += nodes[str(nid)].coordinates
               
        centroid = centroid/len(self.attached_nodes)
        
        return centroid

class Cbush:
    def __init__(self, eid: str, GA: str, GB: str):
        self.eid = int(eid)
        self.GA = int(GA)
        self.GB = int(GB)
        self.attached_nodes = np.array([self.GA,self.GB])

In [3]:
infile_fast = 'FASTENER.dat'

with open(infile_fast) as f:
    lines = [line.strip() for line in f]
    
fast_nodes = {}
cbush = {}

for line in lines:
    if line.startswith('GRID'):
        nid = line[8:16].strip()
        x = line[24:32]
        y = line[32:40]
        z = line[40:48]
        fast_nodes[nid] = Node(nid,x.strip(),y.strip(),z.strip())
    elif line.startswith('CBUSH'):
        eid = line[8:16].strip()
        GA = line[24:32]
        GB = line[32:40]
        cbush[eid] = Cbush(eid,GA.strip(),GB.strip())

In [4]:
for eid in sorted(cbush):
    print(cbush[eid].eid, cbush[eid].attached_nodes)

3000 [3000 3001]


In [5]:
infile_lst = ['TOP.dat', 'BOTTOM.dat']

nodes = {}
cquad4 = {}
    
for infile in infile_lst:

    with open(infile) as f:
        lines = [line.strip() for line in f]

    for line in lines:
        # ADD NODES
        if line.startswith('GRID'):
            nid = line[8:16].strip()
            x   = line[24:32].strip()
            y   = line[32:40].strip()
            z   = line[40:48].strip()

            nodes[nid] = Node(nid, x, y, z)

        # ADD CQUAD4
        elif line.startswith('CQUAD4'):
            eid = line[8:16]
            node_list = [line[24:32],line[32:40],line[40:48],line[40:48]]

            cquad4[eid] = ShellElement(eid)

            for i in range(4):
                cquad4[eid].attached_nodes.append(int(node_list[i].split()[0]))

In [6]:
for eid in sorted(cquad4):
    print(cquad4[eid].eid,cquad4[eid].face_centroid(nodes))

1000 [76.250025 21.600025  0.      ]
1001 [76.2500375 16.8000625  0.       ]
1002 [76.2500375 12.000097   0.       ]
1003 [76.2500375  7.200132   0.       ]
1004 [76.2500125  2.400048   0.       ]
1005 [71.25005  21.600025  0.      ]
1006 [71.2500875 16.800075   0.       ]
1007 [71.2500875 12.000122   0.       ]
1008 [71.2500875  7.200168   0.       ]
1009 [71.2500375  2.400096   0.       ]
1010 [66.250075 21.600025  0.      ]
1011 [66.2501375 16.800075   0.       ]
1012 [66.2501375 12.000122   0.       ]
1013 [66.2501375  7.200168   0.       ]
1014 [66.2500625  2.400096   0.       ]
1015 [61.2501   21.600025  0.      ]
1016 [61.2501875 16.800075   0.       ]
1017 [61.2501875 12.000122   0.       ]
1018 [61.2501875  7.200168   0.       ]
1019 [61.2500875  2.400096   0.       ]
1020 [56.250125 21.600025  0.      ]
1021 [56.2502375 16.800075   0.       ]
1022 [56.2502375 12.000122   0.       ]
1023 [56.2502375  7.200168   0.       ]
1024 [56.2501125  2.400096   0.       ]
1025 [51.25015 

In [8]:
elem_centroids = np.zeros([len(cquad4),3])

for i,eid in enumerate(sorted(cquad4.keys())):
    elem_centroids[i] = cquad4[eid].face_centroid(nodes)

tree = KDTree(elem_centroids,leaf_size=40,metric='euclidean')

tst_dict = {}

for nid in sorted(fast_nodes.keys()):
        
    pnt = np.array([fast_nodes[nid].coordinates])
        
    dist,ind = tree.query(pnt,k=1)

    # if dist[0][0] < 2:
    pierced_elem = []
    for eid in sorted(cquad4.keys()):
        if (cquad4[eid].face_centroid(nodes) == elem_centroids[ind[0]]).all():
            pierced_elem.append(eid)
            
    if not pierced_elem[0].strip() in tst_dict:
        tst_dict[nid] = [ pierced_elem[0].strip(), cquad4[pierced_elem[0]].attached_nodes]
    else:
        tst_dict[nid].extend(pierced_elem[0].strip(), cquad4[pierced_elem[0]].attached_nodes)

    print(f'Fastener nid: {nid}')
    print(f'Fastener node coord.: {pnt[0]}')
    print(f'kdtree found elem. centroid: {elem_centroids[ind[0][0]]}')
    print(f'kdtree dist: {dist[0][0]}')
    print(f'Elem. id: {pierced_elem[0].strip()}')
    print(f'\n')
    
print(f'{tst_dict}')

Fastener nid: 3000
Fastener node coord.: [63.49993 12.00012  0.     ]
kdtree found elem. centroid: [61.2501875 12.000122   0.       ]
kdtree dist: 2.2497425000008855
Elem. id: 1017


Fastener nid: 3001
Fastener node coord.: [63.49993 12.00012  4.     ]
kdtree found elem. centroid: [64.7499125 12.000122   4.       ]
kdtree dist: 1.2499825000015945
Elem. id: 2007


{'3000': ['1017', [1034, 1030, 1029, 1029]], '3001': ['2007', [2042, 2038, 2037, 2037]]}
